# Pre-Process SPEED+ annotations in COCO format

COCO format is easily usable in most popular DL codebases. 

Here, the SPEED+ dataset is converted to COCO format for ease of use in mmcv

## Imports

In [1]:
import os
import time
import json
import random
import numpy as np


from utils import SatellitePoseEstimationDataset
from projection import Projection
from convert2COCO import COCO_SatPose

## Helpers

In [2]:

def load_json(json_path):
    with open(json_path, 'r') as jf:
        d = json.load(jf)
    return d
    
def get_bounding_box(uv_pt,relax_margin = 5):
    u_max = max(uv_pt[0])*(1+relax_margin/100)
    u_min = min(uv_pt[0])*(1-relax_margin/100)
    v_max = max(uv_pt[1])*(1+relax_margin/100)
    v_min = min(uv_pt[1])*(1-relax_margin/100)

    return [u_min, u_max, v_min, v_max]

def process_bounding_boxes(filenames, labels, model_dict, camK, camD):
    list_labels = []

    feat_keys = list(model_dict)
    x_3d = np.ndarray((3,len(feat_keys)))

    for i,key in enumerate(feat_keys):
        x_3d[:,i] = model_dict[key]

    for filename in filenames:
        q = labels[filename]['q']
        r = labels[filename]['r']
        uv_pts = Projection.project_to_image(q, r, camK, camD, r_B=x_3d)
        features = []

        cam_nu = camK[0,2]*2
        cam_nv = camK[1,2]*2

        uv_pts = np.array(uv_pts)

        for idx in range(uv_pts.shape[1]):
            uv = uv_pts[:,idx].tolist()
            #TODO: Visibility makes a difference? 
            # visibility flag v defined as v=0: not labeled (in which case x=y=0), v=1: labeled but not visible, and v=2: labeled and visible
            visibility = 0 if uv[0]> cam_nu or uv[0]<0 or uv[1]>cam_nv or uv[1]<0 else 2
            feature = {"ID": idx, 'Coordinates': uv, 'Visibility': visibility}
            features.append(feature)

        bbox_limits = get_bounding_box(uv_pts)
        bbox_dict = {'filename': filename, 'label':bbox_limits, 'features': features}
        list_labels.append(bbox_dict)

    return list_labels

## Process bounding box and keypoint data

In [3]:
dataset_root_dir = '../speedplus/' # path to speed+'
camera_file = os.path.join(dataset_root_dir, "camera.json")
model_3D_file =  os.path.join(dataset_root_dir, "tango_model_25pt.json")

model3d_data = load_json(model_3D_file)
camera_data = load_json(camera_file)

K = np.array(camera_data['cameraMatrix']) 
D = np.array(camera_data['distCoeffs'])

dataset = SatellitePoseEstimationDataset(root_dir=dataset_root_dir)
labels = dataset.labels

bboxes = {}

for partition in list(dataset.partitions):
    if partition=="validation" or partition=="train":
        filenames = dataset.partitions[partition]
        bbox_anns = process_bounding_boxes(filenames, labels, model3d_data, K, D)
        bboxes[partition] = bbox_anns

## ===================================
#  ATTENTION: Create a subset if needed
## ====================================
create_subset= True

subset_fraction=0.4 # Fraction of original set
valid_partitions = ["train", "validation"]

new_bboxes = {}
if create_subset:
    for partition in valid_partitions:
        
        old_bboxes = bboxes[partition]
        print(f"Dataset size before: {len(old_bboxes)}")
        terminal_idx= int(0.4*len(old_bboxes)) + 1 
        
        random.shuffle(old_bboxes)
        new_bboxes[partition] = old_bboxes[:terminal_idx]
        print(f"Dataset size after: {len(new_bboxes[partition])}")
    
    bboxes=new_bboxes


Dataset size before: 47966
Dataset size after: 19187
Dataset size before: 11994
Dataset size after: 4798


## Convert to COCO

### args

#### 11-point wireframe Model

In [4]:
# tango_KPs_n = 11
# tango_skeleton = [[1,2],[2,3],[3,4],[4,1],[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[8,5],[1,9],[2,10],[4,11]]

# base_args = {
#         "dataset":"SPEED+",
#         "size":(1920,1200),
#         "url": "",
#         "n_keypoints":tango_KPs_n,
#         "skeleton": tango_skeleton,
#         "extension": "jpg",
#         "img_name_prefix": "img",
#         "img_name_suffix": ""
#         }


### Training Set

In [6]:
suffix = "_25pt_subset40_wh"
#TRAIN
partition = "synthetic"
partition_path = f"../speedplus/{partition}"
subset_name = "train"
train_args = base_args
train_args["image_path"] = os.path.join(partition_path,'images')
train_args['data'] = bboxes[subset_name]
output_json_path = os.path.join(partition_path,f'train_coco{suffix}.json')


train_COCO = COCO_SatPose(**train_args)
# train_COCO.write2File(output_json_path)


Processing Images and Annotations
1.0 %
11.0 %
21.0 %
31.0 %
41.0 %
51.0 %
61.0 %
71.0 %
81.0 %
91.0 %
Processed images and annotations!


### Validation Set

In [7]:
partition = "synthetic"
partition_path = f"../speedplus/{partition}"
subset_name = "validation"
val_args = base_args
val_args["image_path"] = os.path.join(partition_path,'images')
val_args['data'] = bboxes[subset_name]
output_json_path = os.path.join(partition_path,f'val_coco{suffix}.json')


val_COCO = COCO_SatPose(**val_args)
# val_COCO.write2File(output_json_path)

Processing Images and Annotations
1.0 %
11.0 %
21.0 %
31.0 %
41.0 %
51.0 %
61.0 %
71.0 %
81.0 %
91.0 %
Processed images and annotations!
